In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
import boto3
import sys
import re
from datetime import datetime
import configparser

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1663652353215_0008,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
Q400Final = spark.sql('''
                    SELECT * 
                        From precogdb.q400512wps where filename = 'ALL ac=VT-SQB dep=AMD arr=DED fn=SG 3761 dt=20220814.csv' 
                ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
Q400_stage3_df = Q400Final.filter(~(F.col("RAD-ALT").like ("*")) 
                                                        & (F.col("RAD-ALT") != " "))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
Q400_stage3_df.createOrReplaceTempView("Q400Final")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
Q400_stage3_df.coalesce(1).write.csv('s3://sjet-datamart-bucket/PoC-Ayush/Test1-512/', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
Q400Interim = Q400_stage3_df.alias('ED').select('ED.FlightID',
                                                            'ED.DesigFlightNumber',
                                                            'ED.FlightNumberARMS',
                                                            'ED.FlightDate',
                                                            'ED.FromCity',
                                                            'ED.ToCity',
                                                            'ED.BlockTime',
                                                            'ED.ECN',
                                                            'ED.EmployeeName',
                                                            'ED.RANK',
                                                            'ED.ChocksOff',
                                                            'ED.ChocksOn',
                                                            'ED.DOMINT',
                                                            'ED.AircraftType',
                                                            'ED.AircraftModel',  
                                                            'ED.Payload',
                                                            'ED.TailNo',
                                                            'ED.Departure',
                                                            'ED.Arrival',
                                                            'ED.FlightNo',
                                                            'ED.Date',
                                                            'ED.Time-In-Secs', 
                                                            'ED.Time-Per-Sec', 
                                                            'ED.BARO-CORRECTED-PALT-DERIVED', 
                                                            'ED.Altitude-BaroL', 
                                                            'ED.HEADING-MAGNETIC', 
                                                            'ED.PITCH-ANGLE', 
                                                            'ED.ROLL-ANGLE',
                                                            F.col('ED.RAD-ALT').cast('Integer').alias('RAD-ALT'), 
                                                            F.lit(F.regexp_replace('ED.WOW-NLG', '\\*', '')).alias('WOW-NLG'), 
                                                            F.lit(F.regexp_replace('ED.WOW-MLG', '\\*', '')).alias('WOW-MLG'),
                                                            'ED.ACCN-NORMAL', 
                                                            'ED.ACCN-LATERAL', 
                                                            'ED.ACCN-LONGITUDINAL', 
                                                            'ED.AIRSPEED-CAS-L', 
                                                            'ED.AFCS-SEL-AIRSPEED', 
                                                            'ED.VERTICAL-SPEED-Derived', 
                                                            'ED.FMS-GROUNDSPEED', 
                                                            'ED.FLAP-CONTROL-POS-Gated', 
                                                            'ED.FLAP-POSITION', 
                                                            'ED.PLA-1', 
                                                            'ED.PLA-2', 
                                                            'ED.TORQUE-1', 
                                                            'ED.TORQUE-2', 
                                                            'ED.CLA-1', 
                                                            'ED.CLA-2', 
                                                            'ED.NP-1', 
                                                            'ED.NP-2', 
                                                            'ED.EGPWS-ALERT-WARNING', 
                                                            'ED.EGPWS-GS-ALERT', 
                                                            F.regexp_replace(F.col('ED.AFCS-GS-MODE'), '\\*', '').alias('AFCS-GS-MODE'),
                                                            'ED.AFCS-LOC-MODE', 
                                                            'ED.AFCS-VOR-MODE', 
                                                            F.regexp_replace(F.col('ED.ILS-GS-DEV-1'), '\\*', '').alias('ILS-GS-DEV-1'),  
                                                            F.regexp_replace(F.col('ED.ILS-GS-DEV-2'), '\\*', '').alias('ILS-GS-DEV-2'),
                                                            'ED.DME-DISTANCE-1', 
                                                            'ED.DME-DISTANCE-2', 
                                                            'ED.VOR-FREQ-1', 
                                                            'ED.VOR-FREQ-2', 
                                                            'ED.FMS-WIND-ANGLE', 
                                                            'ED.FMS-WIND-SPEED', 
                                                            'ED.BRAKE-PRESS-LEFT', 
                                                            'ED.BRAKE-PRESS-RIGHT', 
                                                            'ED.PROP-BETA-RANGE-1', 
                                                            'ED.PROP-BETA-RANGE-2', 
                                                            'ED.AP-ENGAGED', 
                                                            'ED.FileName',
                                                            'ED.LoadedDate',
                                                            'ED.LoadedBy',
                                                            'ED.DataFrameName',
                                                            'ED.year',
                                                            'ED.month')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
Q400Interim.na.fill(0).createOrReplaceTempView("Q400Interim")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
Q400GSD = spark.sql(f'''
                                 SELECT  
                                        'FU 0.8' Limit,
                                        `Time-In-Secs` as `Time-In-Secs-EX`,
                                        `BARO-CORRECTED-PALT-DERIVED` as `BARO-CORRECTED-PALT-DERIVED-EX`,
                                        `RAD-ALT` AS `RAD-ALT-EX` ,
                                        `ILS-GS-DEV-1` `ILS-GS-DEV-1-EX`,
                                        `ILS-GS-DEV-2` `ILS-GS-DEV-2-EX`,
                                        'DeviationBelowG/S' as ExceedanceName,
                                         FN.*

                                FROM (
                                        SELECT 
                                            case
                                                    when
                                                            `Time-In-Secs`<DJ.MinTime
                                                    Then
                                                            'TakeOff'
                                                    when
                                                            `Time-In-Secs` between DJ.MinTime and DJ.MaxTime
                                                    Then
                                                            'Airborne'
                                                    when
                                                            `Time-In-Secs`>DJ.MaxTime
                                                    Then
                                                            'Approach'
                                                    end Phase,
                                            case
                                                when
                                                        case
                                                            when
                                                                `ILS-GS-DEV-1` like 'FU%'
                                                            then
                                                                    CASE
                                                                    WHEN
                                                                            instr(`ILS-GS-DEV-1`,' ')>0
                                                                    THEN
                                                                            SUBSTRING(`ILS-GS-DEV-1`, instr(`ILS-GS-DEV-1`,' ')+1, LENGTH(`ILS-GS-DEV-1`))
                                                                    ELSE
                                                                            NULL
                                                                    END
                                                            END      > 0.8
                                                            AND     `Time-In-Secs` > DJ.MaxTime
                                                Then
                                                        'High'
                                                else
                                                        'Low'
                                                end TriggerCondition1,


                                            case
                                                when
                                                        case
                                                            when
                                                                `ILS-GS-DEV-2` like 'FU%'
                                                            then
                                                                    CASE
                                                                    WHEN
                                                                            instr(`ILS-GS-DEV-2`,' ')>0
                                                                    THEN
                                                                            SUBSTRING(`ILS-GS-DEV-2`, instr(`ILS-GS-DEV-2`,' ')+1, LENGTH(`ILS-GS-DEV-2`))
                                                                    ELSE
                                                                            NULL
                                                                    END
                                                            end   > 0.8
                                                            AND   `Time-In-Secs` > DJ.MaxTime
                                                Then
                                                        'High'
                                                else
                                                        'Low'
                                            END TriggerCondition2,
                                                                        
                                            BJ.*

                                        FROM Q400Interim AS BJ

                            INNER JOIN(

                                        SELECT 

                                                BI.FileName,
                                                BI.`RAD-ALT`,
                                                MIN(`Time-In-Secs`) as MinTime,
                                                MAX(`Time-In-Secs`) as MaxTime

                                        FROM Q400Interim AS BI
                                        INNER JOIN 
                                                    (  SELECT 
                                                            FileName,
                                                            MAX(`RAD-ALT` ) AS MaxRadioHeight
                                                    FROM Q400Interim

                                                    GROUP BY FileName) T1 ON BI.FileName=T1.FileName 
                                                    AND BI.`RAD-ALT`=T1.MaxRadioHeight

                            GROUP BY 
                                    BI.FileName,
                                    BI.`RAD-ALT` ) DJ ON BJ.FileName=DJ.FileName ) FN

                        WHERE FN.Phase='Approach'

                        AND FN.`AFCS-GS-MODE` = 'Active (1)'
                        AND FN.`RAD-ALT` < 1000
                        AND FN.`RAD-ALT` > 200
                        
                        AND ( FN.TriggerCondition1='High' OR
                            FN.TriggerCondition2='High' )
                        AND    FN.`WOW-MLG`='Air (0)' 
                        
                 ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
Q400GSD.createOrReplaceTempView('Q400GSD')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
Q400GSD.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [19]:
Q400GSD.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------------+------------------------------+----------+---------------+---------------+-----------------+--------+-----------------+-----------------+--------+-----------------+----------------+----------+--------+------+---------+------+----------------+-----+--------------------+--------------------+------+------------+-------------+-------+------+---------+-------+--------+--------+------------+------------+---------------------------+--------------+----------------+-----------+----------+-------+-------+-------+-----------+------------+-----------------+--------------+-----------------+----------------------+---------------+----------------------+-------------+-----+-----+--------+--------+-----+-----+----+----+-------------------+--------------+------------+-------------+--------------+------------+------------+--------------+--------------+----------+----------+--------------+--------------+----------------+-----------------+-----------------+-----------------+-------

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------------------+-----------------+-------------------------------+-----------------------------+--------------+-------------------+--------+-----------------+-----------------+--------+-----------------+----------------+----------+--------+------+---------+------+----------------+-----+-------------------+-------------------+------+------------+-------------+-------+------+---------+-------+--------+----------+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+----

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------+----------------+----------+--------+------+---------+------+-------------------+-----+-------------------+-------------------+-------------------+------+------------+-------------+-------+------+---------+-------+--------+----------+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+----------------------------+--------------+--------------+-----------------+-----------------+----------+----------+----------------+-----------

In [30]:
Q400GSD = spark.sql(f'''
                                SELECT  
                                        'FU 1.00' Limit,
                                        `Time-In-Secs` as `Time-In-Secs-EX`,
                                        `BARO-CORRECTED-PALT-DERIVED` as `BARO-CORRECTED-PALT-DERIVED-EX`,
                                        `Radio-Altitude` AS `Radio-Altitude-EX` ,
                                        `GLIDE-PATH-DEV-COPILOT-ABOVE` `GLIDE-PATH-DEV-COPILOT-ABOVE-EX`,
                                        `GLIDE-PATH-DEV-PILOT-ABOVE` `GLIDE-PATH-DEV-PILOT-ABOVE-EX`,
                                        'G/SDeviation' as ExceedanceName,
                                         FN.*

                                FROM (

                                        SELECT 

                                            case
                                                    when
                                                            `Time-In-Secs`<DJ.MinTime
                                                    Then
                                                            'TakeOff'
                                                    when
                                                            `Time-In-Secs` between DJ.MinTime and DJ.MaxTime
                                                    Then
                                                            'Airborne'
                                                    when
                                                            `Time-In-Secs`>DJ.MaxTime
                                                    Then
                                                            'Approach'
                                                    end Phase,
                                            case
                                                when
                                                        case
                                                            when
                                                                `GLIDE-PATH-DEV-COPILOT-ABOVE` like 'FU%'
                                                            then
                                                                    CASE
                                                                    WHEN
                                                                            instr(`GLIDE-PATH-DEV-COPILOT-ABOVE`,' ')>0
                                                                    THEN
                                                                            SUBSTRING(`GLIDE-PATH-DEV-COPILOT-ABOVE`, instr(`GLIDE-PATH-DEV-COPILOT-ABOVE`,' ')+1, LENGTH(`GLIDE-PATH-DEV-COPILOT-ABOVE`))
                                                                    ELSE
                                                                            NULL
                                                                    END
                                                            END      > 1.00
                                                            AND     `Time-In-Secs` > DJ.MaxTime
                                                Then
                                                        'High'
                                                else
                                                        'Low'
                                                end TriggerCondition1,


                                            case
                                                when
                                                        case
                                                            when
                                                                `GLIDE-PATH-DEV-PILOT-ABOVE` like 'FU%'
                                                            then
                                                                    CASE
                                                                    WHEN
                                                                            instr(`GLIDE-PATH-DEV-PILOT-ABOVE`,' ')>0
                                                                    THEN
                                                                            SUBSTRING(`GLIDE-PATH-DEV-PILOT-ABOVE`, instr(`GLIDE-PATH-DEV-PILOT-ABOVE`,' ')+1, LENGTH(`GLIDE-PATH-DEV-PILOT-ABOVE`))
                                                                    ELSE
                                                                            NULL
                                                                    END
                                                            end   > 1.00
                                                            AND   `Time-In-Secs` > DJ.MaxTime
                                                Then
                                                        'High'
                                                else
                                                        'Low'
                                            END TriggerCondition2,
                                                                        
                                            BJ.*

                                        FROM Q400Interim AS BJ

                            INNER JOIN(

                                        SELECT 

                                                BI.FileName,
                                                BI.`Radio-Altitude`,
                                                MIN(`Time-In-Secs`) as MinTime,
                                                MAX(`Time-In-Secs`) as MaxTime

                                        FROM Q400Interim AS BI
                                        INNER JOIN 
                                                    (  SELECT 
                                                            FileName,
                                                            MAX(`Radio-Altitude` ) AS MaxRadioHeight
                                                    FROM Q400Interim

                                                    GROUP BY FileName) T1 ON BI.FileName=T1.FileName 
                                                    AND BI.`Radio-Altitude` =T1.MaxRadioHeight

                            GROUP BY 
                                    BI.FileName,
                                    BI.`Radio-Altitude` ) DJ ON BJ.FileName=DJ.FileName ) FN

                        WHERE FN.Phase='Approach'
                        AND FN.`AFCS-Glideslope-Active` = 'Active (1)'
                        AND FN.`Radio-Altitude` < 1000
                        AND FN.`Radio-Altitude` > 200 
                        AND ( FN.TriggerCondition1='High' OR
                            FN.TriggerCondition2='High' )
                        AND    FN.`Main-WOW`='Air (0)' 
                        
                 ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
Q400GSD.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [28]:
Q400GSD = spark.sql(f'''
                                SELECT  
                                        'FU 1.00' Limit,
                                        `Time-In-Secs` as `Time-In-Secs-EX`,
                                        `BARO-CORRECTED-PALT-DERIVED` as `BARO-CORRECTED-PALT-DERIVED-EX`,
                                        `Radio-Altitude` AS `Radio-Altitude-EX` ,
                                        `GLIDE-PATH-DEV-COPILOT-ABOVE` `GLIDE-PATH-DEV-COPILOT-ABOVE-EX`,
                                        `GLIDE-PATH-DEV-PILOT-ABOVE` `GLIDE-PATH-DEV-PILOT-ABOVE-EX`,
                                        'G/SDeviation' as ExceedanceName,
                                         FN.*

                                FROM (

                                        SELECT 

                                            case
                                                    when
                                                            `Time-In-Secs`<DJ.MinTime
                                                    Then
                                                            'TakeOff'
                                                    when
                                                            `Time-In-Secs` between DJ.MinTime and DJ.MaxTime
                                                    Then
                                                            'Airborne'
                                                    when
                                                            `Time-In-Secs`>DJ.MaxTime
                                                    Then
                                                            'Approach'
                                                    end Phase,
                                            case
                                                when
                                                        case
                                                            when
                                                                `GLIDE-PATH-DEV-COPILOT-ABOVE` like 'FU%'
                                                            then
                                                                    CASE
                                                                    WHEN
                                                                            instr(`GLIDE-PATH-DEV-COPILOT-ABOVE`,' ')>0
                                                                    THEN
                                                                            SUBSTRING(`GLIDE-PATH-DEV-COPILOT-ABOVE`, instr(`GLIDE-PATH-DEV-COPILOT-ABOVE`,' ')+1, LENGTH(`GLIDE-PATH-DEV-COPILOT-ABOVE`))
                                                                    ELSE
                                                                            NULL
                                                                    END
                                                            END      > 1.00
                                                            AND     `Time-In-Secs` > DJ.MaxTime
                                                Then
                                                        'High'
                                                else
                                                        'Low'
                                                end TriggerCondition1,


                                            case
                                                when
                                                    case
                                                        when
                                                                `GLIDE-PATH-DEV-PILOT-ABOVE` like 'FU%'
                                                        then
                                                                CASE
                                                                WHEN
                                                                        instr(`GLIDE-PATH-DEV-PILOT-ABOVE`,' ')>0
                                                                THEN
                                                                        SUBSTRING(`GLIDE-PATH-DEV-PILOT-ABOVE`, instr(`GLIDE-PATH-DEV-PILOT-ABOVE`,' ')+1, LENGTH(`GLIDE-PATH-DEV-PILOT-ABOVE`))
                                                                ELSE
                                                                        NULL
                                                                END
                                                        end   > 1.00
                                                        AND   `Time-In-Secs` > DJ.MaxTime
                                                Then
                                                        'High'
                                                else
                                                        'Low'
                                            END TriggerCondition2,
                                                                        
                                            BJ.*

                                        FROM Q400Interim AS BJ

                            INNER JOIN(

                                        SELECT 

                                                BI.FileName,
                                                BI.`Radio-Altitude`,
                                                MIN(`Time-In-Secs`) as MinTime,
                                                MAX(`Time-In-Secs`) as MaxTime

                                        FROM Q400Interim AS BI
                                        INNER JOIN 
                                                    (  SELECT 
                                                            FileName,
                                                            MAX(`Radio-Altitude` ) AS MaxRadioHeight
                                                    FROM Q400Interim

                                                    GROUP BY FileName) T1 ON BI.FileName=T1.FileName 
                                                    AND BI.`Radio-Altitude` =T1.MaxRadioHeight

                            GROUP BY 
                                    BI.FileName,
                                    BI.`Radio-Altitude` ) DJ ON BJ.FileName=DJ.FileName ) FN

                        WHERE FN.Phase='{Phase}'
                        AND FN.`Radio-Altitude` <{RadioHeight}
                        AND ( FN.TriggerCondition1='{TriggerConditionL}' OR
                            FN.TriggerCondition2='{TriggerConditionR}' )
                        AND    FN.`Main-WOW`='Air (0)' 
                        
                    
                ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
FourthQuery.createOrReplaceTempView("E")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
FourthQuery.show(5, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------------------+-----------------+-------------------------------+-----------------------------+--------------+-------------+-----+-----------------+-----------------+--------+-----------------+----------------+----------+--------+------+---------+---+------------+----+---------+--------+------+------------+-------------+-------+------+---------+-------+--------+----+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+-----

In [30]:
FifthQuery = spark.sql(''' select * FROM E WHERE TriggerCondition1 = 'High' OR TriggerCondition2 = 'High' and  `AFCS-Autopilot-Engaged` = 'Engaged (1)' and Phase = 'Approach' ''')

FifthQuery.createOrReplaceTempView("F")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
FifthQuery.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0